# TODO
- Check if the series needs / benefits from a BoxCox transform

In [ ]:
library(forecast)

In [ ]:
loadData <- function(dataFolder) {
    files <- list.files(dataFolder)
    data <- list()
    for(file in files) {    
        df <- read.csv(paste0(dataFolder, "/", file), stringsAsFactors=F)    
        minYear <- min(df$Year)
        complaintType <- substr(file,1,(nchar(file))-4)    
        tsObject <- ts(df$Complaints, start=c(minYear, 1), frequency = 12)
        data[[complaintType]] <- tsObject
    }
    data
}
data <- loadData("../../data/topNComplaints")

In [ ]:
series <- data[["Non Burning of Street Lights"]]

In [ ]:
tsdisplay(series)

In [ ]:
# data before 2012 are too few to consider
series <- window(series, start=c(2012, 1), end=c(2016, 6))
tsdisplay(series)

## Cleaning up data 

Although this data looks like it doesn't have any outliers, let's take a look at where the potential extreme values are

In [ ]:
plot(series, col="red", lty=2)
lines(tsclean(series), lty=1)
legend("topright", col=c("red", "black"), lty=c(2,1), legend=c("Original", "Cleaned"))

Taking a call here that the data doesn't contain any outliers, so we're leaving the data as it is

## Decomposition

In [ ]:
# first try a static seasonal component
plot(stl(series, s.window="periodic"))

The trend component is the most significant here, so the series probably needs some differencing. Strangely, there is also a seasonal component. Let's take varying s.window to see if changes over time. 

In [ ]:
old.par <- par(mfrow=c(2, 2), mar=c(3,3,3,3))
plot(stl(series, s.window=3)$time.series[, 1], main="Seasonal Component with s.window = 3")
plot(stl(series, s.window=6)$time.series[, 1], main="Seasonal Component with s.window = 6")
plot(stl(series, s.window=10)$time.series[, 1], main="Seasonal Component with s.window = 10")
plot(stl(series, s.window=12)$time.series[, 1], main="Seasonal Component with s.window = 12")
par(old.par)

Looks like the seasonal component is there, but $s.window=3$ suggests that it is not as significant

In [ ]:
seasonal <- stl(series, s.window="periodic")$time.series[, 1] # change s.window
plot(seasonal, col="grey")
month <- 11 # change this to month you want
for(i in 2012:2016) {    
    abline(v=(month-1)/12 + i, lty=2)
}

**Looks like it peaks in November. **

Let us then do a seasonal adjustment of the data. All further analysis should be done on this data

In [ ]:
stl.fit <- stl(series, s.window="periodic")
series.adj <- seasadj(stl.fit)
tsdisplay(series.adj)

## Forecasting
### ARIMA models - estimating p, d, q

First, let us estimate $d$. This is done by looking at the ACF of the data.

In [ ]:
Acf(series.adj)

In [ ]:
# the above series is a classic example of a series that requires a diff of order 1, 
# so let's try that out and take a look at the Acf to see if it is overdifferenced
series.diff <- diff(series.adj, lag=1, differences = 1)
tsdisplay(series.diff)

In [ ]:
# the series looks good!
# let's take a look at the standard deviation as well
sd(series.adj)

In [ ]:
sd(series.diff)

In [ ]:
# looks good - it has decreased. Since stationary series return to the mean, let's take a look at that as well
plot(series.diff, col="grey")
# a 2x4 MA
lines(ma(ma(series.diff, order=2), order=4))
abline(mean(series.diff), 0, col="blue", lty=2)

In [ ]:
# let's verify once wheather d=1
ndiffs(series.adj)

Next, we need to estimate p and q. To do this, we take a look at the PACF of the data. Note that this analysis is done on the differenced data. If we decide to fit a model with d=0, then we need to perform this analysis for the un-differenced data as well

In [ ]:
# for d=0
Pacf(series.adj)

In [ ]:
# looks like a AR(1) and a MA(5) process
# take a look at the d=1
Pacf(series.diff)

In [ ]:
# this looks like a MA(11) and a AR(4) process

#### Building candidate models

In [ ]:
modelArima <- function(series, order, h, testData = NULL) {
    fit <- Arima(series, order=order)
    print(summary(fit))
    predictions <- forecast(fit, h)
    # compute max and min y
    min.yvalue <- min(min(series), min(testData))
    max.yvalue <- max(max(series), max(testData))
    
    plot(predictions, ylim=c(min.yvalue, max.yvalue))
    if(!is.null(testData)) {
        lines(testData, col="red", lty=2)
        print(accuracy(predictions, testData))
    }
    # check if residuals looklike white noise
    Acf(residuals(fit), main="Residuals")
    # portmantaeu test
    print(Box.test(residuals(fit), lag=24, fitdf=4, type="Ljung"))
}

In [ ]:
# split the series into a test and a train set
series.train <- window(series.adj, end=c(2015, 6))
series.test <- window(series.adj, start=c(2015, 7))

In [ ]:
# with d=0, order=(1, 0, 5)
modelArima(series.train, c(1, 0, 5), length(series.test), series.test)

In [ ]:
# with d=1, order=(4, 1, 11)
modelArima(series.train, c(4, 1, 11), length(series.test), series.test)

In [ ]:
# fiddle with p and q, with d=1
modelArima(series.train, c(5, 1, 11), length(series.test), series.test)

In [ ]:
modelArima(series.train, c(4, 1, 12), length(series.test), series.test)

In [ ]:
modelArima(series.train, c(4, 1, 10), length(series.test), series.test)

In [ ]:
modelArima(series.train, c(3, 1, 11), length(series.test), series.test)

## Exponential Smoothing

In [ ]:
#Non Burning of street lights
series <- window(series, start = c(2012,4), end = c(2016,6))
stl.fit <- stl(series, s.window=8)
series.adj <- seasadj(stl.fit)
seasonal <- stl.fit$time.series[, 1]
seasonal_train <- stl(window(series, end = c(2015,6)), s.window = 8)[[1]][,1]
#tsdisplay(series.adj)
plot(seasonal)
plot(seasonal_train)

In [ ]:
stl.fit <- stl(series, s.window="periodic")
series.adj <- seasadj(stl.fit)
seasonal <- stl.fit$time.series[, 1]
seasonal_train <- stl(window(series, end = c(2015,6)), s.window = "periodic")[[1]][,1]
#tsdisplay(series.adj)
plot(seasonal)
plot(seasonal_train)

In [ ]:
seasonal
seasonal_train

#### Note: From the plot and the data points, it looks like the seasonal component varies for recent time period, which is not clearly captured by the trianing dataset, which in turn may affect the future prediction if we consider only the seasonality of the training dataset.

In [ ]:
## Function for finding the average of seasonal components
period_stat <- function(ts_data_in, type = 1, start_value, years){
#type 1: sum
#type 2: mean

freq <- frequency(ts_data_in)
len <- length(ts_data_in)

freq_vector <- numeric(0)
freq_sum <- numeric(0)
vec <- numeric(0)
sum_vec <- numeric(0)

start_val <- start(ts_data_in)

ts_data_in <- c(rep(NA,start_val[2] - 1),ts_data_in)

max_limit <- ceiling(len/freq)
    for(i in 1:max_limit){
    
    vec <- ts_data_in[(((i-1)*freq)+1):(((i-1)*freq)+freq)]
    freq_vector <- as.numeric(!is.na(vec))
    vec[is.na(vec)] <- 0
    
    if(i == 1){
    sum_vec <- vec
    freq_sum <- freq_vector
    }else{
    sum_vec <- sum_vec + vec
    freq_sum <- freq_sum + freq_vector
    }
    }

final_ts <- numeric(0)
if(type == 1)
{
    final_ts <- sum_vec
}else if(type == 2) {

    final_ts <- (sum_vec/freq_sum)
} else {
    stop("Invalid type")
}


return(ts(rep(final_ts,years),frequency = freq, start = start_value ))

}

In [ ]:
#Adjust the negative values in the ts data
es_series <- series.adj
min_ts_value <- min(es_series)

bias_value <- (-1*min_ts_value) + 1
ES_series <- es_series+ bias_value
#plot(ES_series)
ES_series

train_data <- window(ES_series, end=c(2015, 6))
test_data <- window(ES_series, start=c(2015, 7))

In [ ]:
#Getting the mean value from the seasonal components for the data set and not for the training set alone.
#Need to adjust based on the input from Suchana.

seasonal_mean <- period_stat(seasonal,2,c(2012,1),years = 7)

In [ ]:
#Preprocessing data. Removing 0 from the data
train_data[train_data==0]=0.01 

#Fitting a model with ets function

ets1 = ets(train_data)
summary(ets1)
plot(forecast(ets1))
lines(test_data, col = "red")

In [ ]:
#Ljung Box test - One of the checks to perform stationarity of TS data
Box.test(ets1$residuals, lag = 20, type = "Ljung-Box")
p_value <- Box.test(ets1$residuals, lag = 20, type = "Ljung-Box")$p.value
Acf(ets1$residuals)

## Finding the best fit for exponential smoothing

In [ ]:
all_types = c("ANN","AAN","AAA","ANA","MNN","MAN","MNA","MAA","MMN","MNM","MMM","MAM")
forecast_values = 12
# For eg: AAA -> additive level, additive trend and additive seasonality
# ANN -> No trend or seasonality

In [ ]:
all_fit <- list()
test_models <- list()

print("Fitting various models: ")
for (bool in c(TRUE,FALSE)){
    for (model_type in all_types){

        if(bool & substr(model_type,2,2)=="N"){
            next
        }
    test_model = ets(train_data, model = model_type,damped = bool)
    #Box.test(test_model$residuals, lag = 20, type = "Ljung-Box")$p.value
    all_fit[[paste0("ETS Model: ",model_type,", Damped: ",bool)]][1] <- accuracy(test_data, forecast.ets(test_model,h=forecast_values)$mean )[5]
    all_fit[[paste0("ETS Model: ",model_type,", Damped: ",bool)]][2] <- 100*(Box.test(test_model$residuals, lag = 20, type = "Ljung-Box")$p.value)
    
        test_models[[paste0("ETS Model: ",model_type,", Damped: ",bool)]] <- test_model

        print(test_model$method)
        print(accuracy(test_data, forecast.ets(test_model,h=forecast_values)$mean )[5])
        print("")

        #Excluding the models which has auto correlated residuals @ 10% significance level

    }
}

In [ ]:
#Finding the best fit
proper_models <- all_fit
    if(length(proper_models)==0){
        print("None of the model satisfies - Ljung-Box test; Model with least 3 p values taken")
        p_values <- sapply(all_fit, function(x)x[2])
        proper_models <- all_fit[order(p_values)][1:3]
    }

    best_mape <- min(sapply(proper_models,function(x)x[1]))
    best_model <- names(which.min(sapply(proper_models,function(x)x[1])))

    print(paste0("Complaint: ",names(TS_data)))
    print(paste0("Best Model:",best_model))
    print(paste0("Best Mape: ",best_mape))

In [ ]:
#Finding top n fits
top_models <- c()
Top_n <- 3

if(length(proper_models)<3){Top_n <- length(proper_models)}

top_mape_val <- proper_models[order(sapply(proper_models, function(x)x[1]))][1:Top_n]
top_models <- names(top_mape_val)

In [ ]:
top_mape_val
seasonal_mean

In [ ]:
plot(ES_series,col = "black")
lines(test_data, col = "blue")
lines(forecast.ets(test_models[[top_models[1]]],h=12)$mean, col = "red") #Top model
lines(forecast.ets(test_models[[top_models[2]]],h=12)$mean, col = "green") #Top second model
lines(forecast.ets(test_models[[top_models[3]]],h=12)$mean, col = "yellow") #Top third model

#Observation: Unusual peak at December'15. To check if it is an anomaly

### Getting back the original data

In [ ]:
#Adding the bias value which was added to overcome the negative values
ES_series_bias <- ES_series - bias_value
test_series_bias <- test_data - bias_value
forecast1_bias <- forecast.ets(test_models[[top_models[1]]],h=12)$mean - bias_value
forecast2_bias <- forecast.ets(test_models[[top_models[2]]],h=12)$mean - bias_value
forecast3_bias <- forecast.ets(test_models[[top_models[3]]],h=12)$mean - bias_value

#Adding back the seasonal value from stl decomposition
ES_value <- ES_series_bias + seasonal
test_series <- test_series_bias + seasonal

forecast1 <- forecast1_bias + seasonal_mean
forecast2 <- forecast2_bias + seasonal_mean
forecast3 <- forecast3_bias + seasonal_mean


accuracy(test_models[[top_models[1]]])
accuracy(test_models[[top_models[2]]])
accuracy(test_models[[top_models[3]]])

In [ ]:
#Checking the MAPE values with original data
print(paste0("Top model: ", top_models[1]))
accuracy(forecast1,test_series)
print(paste0("Top model: ", top_models[2]))
accuracy(forecast2,test_series)
print(paste0("Top model: ", top_models[3]))
accuracy(forecast3,test_series)

#accuracy(test_data, forecast.ets(test_models[[top_models[3]]],h=12)$mean )

In [ ]:
#Ljung Box test - One of the checks to perform stationarity of TS data
#Top model
print(top_models[1])
Box.test(test_models[[top_models[1]]]$residuals, lag = 20, type = "Ljung-Box")
p_value <- Box.test(test_models[[top_models[1]]]$residuals, lag = 20, type = "Ljung-Box")
Acf(test_models[[top_models[1]]]$residuals)

In [ ]:
#Ljung Box test - One of the checks to perform stationarity of TS data
#Top second model
print(top_models[2])
Box.test(test_models[[top_models[2]]]$residuals, lag = 20, type = "Ljung-Box")
p_value <- Box.test(test_models[[top_models[2]]]$residuals, lag = 20, type = "Ljung-Box")
Acf(test_models[[top_models[2]]]$residuals)

In [ ]:
#Ljung Box test - One of the checks to perform stationarity of TS data
#Top Third model
print(top_models[3])
Box.test(test_models[[top_models[3]]]$residuals, lag = 20, type = "Ljung-Box")
p_value <- Box.test(test_models[[top_models[3]]]$residuals, lag = 20, type = "Ljung-Box")
Acf(test_models[[top_models[3]]]$residuals)

#### Residual output: From the residual plot it is seen that even though first two models have lesser MAPE values, it is the third model "ETS Model: ANN, Damped: FALSE", which has an acceptable residual plot. From the plots of the other two, it is seen that they have high autocorrelation.

In [ ]:
plot(ES_value,col = "black") #Original data set
lines(test_series, col = "blue") #Original test data
lines(test_series_bias + seasonal_mean, col = "black") #Deseasonlised data with average seasonal component applied
lines(forecast1, col = "red") #Top model
lines(forecast2, col = "green") #Top second model
lines(forecast3, col = "yellow") #Top third model

#### Observation: The forecast seems to be a resonably good fit from the plot. Eventhough all the points cannot be captured accurately, the re-seasonalising part seems to be effective which gets back the plot, which is closer to the reality.